In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import dataset
import model
import torch.nn.functional as F
import cv2

In [ ]:
# 각종 path및 하이퍼 파라미터 설정
data_path = 'C:\\Users\\USER\\Desktop\\GSH_CRP\\codes\\rock_sci_paper\\data\\ro_sci_pa_heo'
save_path = 'C:\\Users\\USER\\Desktop\\GSH_CRP\\codes\\rock_sci_paper\\model_para'
batch_size = 16
seed = 0
mode = 'hr'

In [ ]:
# seed 설정
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

In [ ]:
test_dataset = dataset.RockScissorsPaper(
    transform=test_transform,
    path = data_path,
    mode = 'test'
)

In [ ]:
model = model.ResNet18(num_classes=3)
model = model.to(device)

model.load_state_dict(torch.load(os.path.join(save_path, 'student.pth')))

In [ ]:
index = 12
sample, label = test_dataset[index]

print(sample.shape)
h,w = sample.shape[-2], sample.shape[-1]
sample = F.interpolate(sample.unsqueeze(0), (h//8, w//8))
sample = F.interpolate(sample, (h,w))
sample = sample.squeeze()
print(sample.shape)

In [ ]:

sample2 = sample.permute(1,2,0)
# sample2 = cv2.resize(sample2.numpy(), dsize=(32,32), interpolation=cv2.INTER_CUBIC)
print(sample2.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(sample2)
plt.show()

In [ ]:
sample = sample.unsqueeze(0).to(device)

output, f4, _, _, _ = model(sample)

h, w = int(f4.shape[-1]**0.5), int(f4.shape[-1]**0.5)
weights = list(model.parameters())[-2].data

weight = weights[label]
weight = weight.unsqueeze(0).unsqueeze(0)
print(weight.shape)
print(f4.shape)
cam = torch.bmm(weight, f4)
cam = torch.reshape(cam, (1, h, w))
cam = cam - torch.min(cam)
cam = cam / torch.max(cam)
cam = cam.unsqueeze(1)
cam = F.interpolate(cam, size=(256,256))
print(cam.shape)

In [ ]:
cam = cam.detach().cpu().numpy()
plt.imshow(cam[0].reshape(256,256,1))
plt.show()